In [1]:
from symred.symplectic import PauliwordOp
from symred.projection.cs_vqe import CS_VQE

In [496]:
H_full = {
 # noncon
 'IIYI': 0.6,
 'XYXI': 0.7,
 'XZXI': 0.7,
 'XZZI': 0.6,
 'YXYI': 0.1,
#  'ZYZI': 0.6,
#  'ZZXI': 0.1,
 'ZZZI': 0.7,
 'IIIZ': 0.5,
    
 # con
 'XXXI': 0.1,
 'XXYI': 0.5,
 'XXZI': 0.2,
 'YXXI': 0.2,
#  'YYXI': 0.2,
 'YYZI': 0.2,
 'YZXI': 0.1,
 'ZYYI': 0.1
      }

In [1091]:
H = PauliwordOp(H_full)
H

0.600+0.000j IIYI +
0.700+0.000j XYXI +
0.700+0.000j XZXI +
0.600+0.000j XZZI +
0.100+0.000j YXYI +
0.700+0.000j ZZZI +
0.500+0.000j IIIZ +
0.100+0.000j XXXI +
0.500+0.000j XXYI +
0.200+0.000j XXZI +
0.200+0.000j YXXI +
0.200+0.000j YYZI +
0.100+0.000j YZXI +
0.100+0.000j ZYYI

In [4]:
cs_vqe_obj = CS_VQE(H)

In [5]:
# H_con
cs_vqe_obj.contextual_operator

0.100+0.000j XXXI +
0.200+0.000j YXXI +
0.100+0.000j YZXI +
0.200+0.000j XXZI +
0.500+0.000j XXYI +
0.200+0.000j YYZI +
0.100+0.000j ZYYI

In [6]:
# H_noncon
cs_vqe_obj.noncontextual_operator

0.700+0.000j XZXI +
0.700+0.000j XYXI +
0.600+0.000j IIYI +
0.100+0.000j YXYI +
0.600+0.000j XZZI +
0.700+0.000j ZZZI +
0.500+0.000j IIIZ

In [7]:
#
cs_vqe_obj.decomposed

{'symmetry': 0.500+0.000j IIIZ,
 'clique_0': 0.600+0.000j IIYI +
 0.100+0.000j YXYI,
 'clique_1': 0.700+0.000j XYXI +
 0.600+0.000j XZZI,
 'clique_2': 0.700+0.000j XZXI +
 0.700+0.000j ZZZI}

In [8]:
r_vec = cs_vqe_obj.clique_operator.coeff_vec
print(r_vec)

[-0.25318281 -0.65828113 -0.70891778]


In [9]:
# A(r)
cs_vqe_obj.clique_operator

-0.253 IIYI +
-0.658 XYXI +
-0.709 XZXI

In [10]:
# q_vec G
cs_vqe_obj.symmetry_generators

-1 YIYI 
 1 IXYI 
-1 IIIZ

In [11]:
# E_noncon
cs_vqe_obj.noncontextual_energy

-2.474841765808721

In [12]:
# term being reduced to via UP
cs_vqe_obj.C0

1.000+0.000j IIYI

In [13]:
# unitary partitioning rots
cs_vqe_obj.unitary_partitioning_rotations

[(1.000-0.000j XYZI, -4.34521811438243),
 (1.000-0.000j XZZI, (0.7879625990338508-0j))]

In [14]:
LCU_rots , Ps = cs_vqe_obj.clique_operator.gen_LCU()

print(f'term reducing too: {Ps}')
print()
print('LCU:')
LCU_rots

term reducing too: 1.000+0.000j IIYI

LCU:


0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [15]:
stabs = PauliwordOp(['YIYI', 'IXYI', 'IIIZ'], [1,1,1])
H_reduced = cs_vqe_obj.project_onto_subspace(stabilizers= stabs,
                                             enforce_clique_operator= False)
H_reduced

-0.500+0.000j I +
-1.400+0.000j X +
-1.300+0.000j Z +
0.500+0.000j Y

In [16]:
cs_vqe_obj.stab_qubit_indices

array([0, 1, 3])

In [17]:
cs_vqe_obj.rotated_stabilizers

-1 ZIII 
 1 IZII 
-1 IIIZ

In [18]:
P * P_rot - P_rot*P

NameError: name 'P' is not defined

In [ ]:
P_rot.commutes(P)

In [188]:
LCU_rot_list = list(LCU_rots)
I_term_LCU = LCU_rot_list[0]
non_I_terms = LCU_rot_list[1:]

empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]
for j, P_rot in enumerate(non_I_terms):
    for P in H:

        dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
        if P.commutes(P_rot):
            #vi = ci_di_di*
            new_const = dI2 + dj_djstar
            ci_dI2_vi_Pi = P.multiply_by_constant(new_const)
            rot_H+= ci_dI2_vi_Pi
        else:
            new_const = dI2 - dj_djstar
            ci_dI2_vi_Pi = P.multiply_by_constant(new_const)
            rot_H-= ci_dI2_vi_Pi
        
        for k, P_rot2 in enumerate(non_I_terms):
            if j!=k:
                if not P_rot2.commutes(P):                    
#                     # ci_dj_PjkPi
#                     ci_dj_PjkPi = P_rot2 * P 
#                      # 2dI_ci_dj_PjkPi
#                     rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)

                    # ci_dj_PjkPi
                    neg_dI_ci_dj_PjkPi = (P * P_rot2).multiply_by_constant(-1*I_term_LCU.coeff_vec[0])
                    dI_ci_dj_PiPijk = (P_rot2 * P).multiply_by_constant(I_term_LCU.coeff_vec[0])
                     # 2dI_ci_dj_PjkPi
                    print('t', dI_ci_dj_PiPijk +  neg_dI_ci_dj_PjkPi)
                    rot_H+= (neg_dI_ci_dj_PjkPi+dI_ci_dj_PiPijk)


In [284]:
LCU_rot_list = list(LCU_rots)
I_term_LCU = LCU_rot_list[0]
non_I_terms = LCU_rot_list[1:]

empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]
for j, P_rot in enumerate(non_I_terms):
    for P in H:

        dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
        if P.commutes(P_rot):
            #vi_Pi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
            dI2_ci_Pi = P.multiply_by_constant(dI2)
            rot_H+= (dI2_ci_Pi + vi_Pi)
        else:
            #vi_Pi = ci_di_di*
            neg_vi_Pi = P.multiply_by_constant(-1*P_rot.coeff_vec[0])
            dI2_ci_Pi = P.multiply_by_constant(dI2)
            rot_H+= (neg_vi_Pi - dI2_ci_Pi)


In [285]:
rot_H

0.137+0.000j XXXI +
-0.149-0.224j YXXI +
0.203-0.406j XZXI +
0.236-0.377j XYXI +
0.034-0.054j YZXI +
-0.149-0.224j XXZI +
-0.448-0.671j IIYI +
0.687+0.000j XXYI +
-0.075-0.112j YXYI +
0.202-0.323j XZZI +
0.203-0.406j ZZZI +
0.067-0.108j YYZI +
0.029-0.058j ZYYI +
0.687+0.000j IIIZ

In [1153]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)

        for l in range(0,len(non_I_terms)):
            Plk = non_I_terms[l]
            if (P.commutes(Plk) and  (not P.commutes(P_rot))):
                Pjk_Plk_Pi =  Plk*P_rot*P
                rot_H+= Pjk_Plk_Pi
                
            if (not P.commutes(Plk) and   P.commutes(P_rot)):
                Pjk_Plk_Pi =  P_rot*Plk*P
                rot_H+= Pjk_Plk_Pi
            
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1152]:
out1 = LCU_rots * H * LCU_rots.conjugate

out1 - rot_H

In [1135]:
LCU_rots * H * LCU_rots.conjugate

-0.395+0.000j IXII +
0.100+0.000j XXXI +
-0.132+0.000j ZIII +
0.496+0.000j YIII +
-0.122+0.000j YXXI +
0.132+0.000j IZII +
-0.142+0.000j IYII +
0.217+0.000j XZXI +
0.251+0.000j XYXI +
0.066+0.000j ZZXI +
0.042+0.000j YZXI +
-0.071+0.000j ZYXI +
0.062+0.000j YYXI +
-0.051+0.000j XXZI +
-1.109+0.000j IIYI +
0.500+0.000j XXYI +
0.066+0.000j ZXYI +
-0.025+0.000j YXYI +
0.252+0.000j XZZI +
0.375+0.000j XYZI +
0.229+0.000j ZZZI +
-0.125+0.000j YZZI +
-0.437+0.000j ZYZI +
0.084+0.000j YYZI +
-0.069+0.000j ZZYI +
0.174+0.000j ZYYI +
0.500+0.000j IIIZ

In [1124]:
# print(rot_H.cleanup().n_terms)
rot_H

-0.395+0.000j IXII +
0.100+0.000j XXXI +
-0.132+0.000j ZIII +
0.496+0.000j YIII +
-0.122+0.000j YXXI +
0.132+0.000j IZII +
-0.142+0.000j IYII +
0.436+0.000j XZXI +
0.470+0.000j XYXI +
0.066+0.000j ZZXI +
0.042+0.000j YZXI +
-0.071+0.000j ZYXI +
0.031+0.000j YYXI +
-0.051+0.000j XXZI +
-1.109+0.000j IIYI +
0.500+0.000j XXYI +
0.066+0.000j ZXYI +
-0.025+0.000j YXYI +
0.252+0.000j XZZI +
0.187+0.000j XYZI +
0.229+0.000j ZZZI +
-0.062+0.000j YZZI +
-0.219+0.000j ZYZI +
0.084+0.000j YYZI +
-0.100+0.000j ZZYI +
0.174+0.000j ZYYI +
0.500+0.000j IIIZ

In [192]:
H

0.600+0.000j IIYI +
0.700+0.000j XYXI +
0.700+0.000j XZXI +
0.600+0.000j XZZI +
0.100+0.000j YXYI +
0.700+0.000j ZZZI +
0.500+0.000j IIIZ +
0.100+0.000j XXXI +
0.500+0.000j XXYI +
0.200+0.000j XXZI +
0.200+0.000j YXXI +
0.200+0.000j YYZI +
0.100+0.000j YZXI +
0.100+0.000j ZYYI

In [172]:
LCU_rots * H * LCU_rots.conjugate

-0.395+0.000j IXII +
0.100+0.000j XXXI +
-0.132+0.000j ZIII +
0.496+0.000j YIII +
-0.122+0.000j YXXI +
0.132+0.000j IZII +
-0.142+0.000j IYII +
0.217+0.000j XZXI +
0.251+0.000j XYXI +
0.066+0.000j ZZXI +
0.042+0.000j YZXI +
-0.071+0.000j ZYXI +
0.062+0.000j YYXI +
-0.051+0.000j XXZI +
-1.109+0.000j IIYI +
0.500+0.000j XXYI +
0.066+0.000j ZXYI +
-0.025+0.000j YXYI +
0.252+0.000j XZZI +
0.375+0.000j XYZI +
0.229+0.000j ZZZI +
-0.125+0.000j YZZI +
-0.437+0.000j ZYZI +
0.084+0.000j YYZI +
-0.069+0.000j ZZYI +
0.174+0.000j ZYYI +
0.500+0.000j IIIZ

In [26]:
P

0.100+0.000j ZYYI

In [27]:
P_rot

0.000+0.539j XYZI

In [29]:
P.anticommutator(P_rot)

0.000-0.108j YIXI

In [ ]:
LCU_rot_list = list(LCU_rots)
I_term_LCU = LCU_rot_list[0]
non_I_terms = LCU_rot_list[1:]

empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]
for j, P_rot in enumerate(non_I_terms):
    for P in H:
        ci_dI2 = P.coeff_vec[0]*dI2
        new_const = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(new_const)
            # ci_dI2_vi_Pi
            rot_H+= vi_Pi.multiply_by_constant(ci_dI2)
        else:
            vi_Pi = P.multiply_by_constant(-1*new_const)
            # ci_dI2_vi_Pi
            rot_H+= vi_Pi.multiply_by_constant(ci_dI2)
        
        for k, P_rot2 in enumerate(non_I_terms):
            if j!=k:
                if not P_rot2.commutes(P):                    
                    # ci_dj_PjkPi
                    ci_dj_PjkPi = P_rot2 * P 
                    
                     # 2dI_ci_dj_PjkPi
                    rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)


In [213]:
test = PauliwordOp({'XXX':1})
test2 = PauliwordOp({'XXY':1})


test3 = test*test2

print(test3)
print(test3.conjugate)

0.000+1.000j IIZ
0.000-1.000j IIZ


In [212]:
test4 = test2*test
print(test4)

test3.conjugate == test4


0.000-0.500j IIZ


True

In [222]:
test = PauliwordOp({'XXX':1})
test2 = PauliwordOp({'XXY':1})

# op = test+test2

# op2 = test.conjugate + test2.conjugate

# op2 == op.conjugate

op3 = test*test2 
op4 = test2*test

op3 == op4.conjugate

True

In [286]:
LCU_rots

0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [347]:
LCU_rots * PauliwordOp({'XXXX':0.5, 'XXXI':0.25}) * LCU_rots.conjugate

0.250+0.000j XXXI +
0.500+0.000j XXXX

In [309]:
0.611*0.611

0.37332099999999996

In [395]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
H =PauliwordOp({'XXXX':0.5, 'XXXI':0.25}) # PauliwordOp({'XXXX': 0.3})
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            new_const =  dj_djstar
            vi_Pi = P.multiply_by_constant(new_const)
            ci_dI2_Pi= P.multiply_by_constant(two_dI)
            rot_H+= vi_Pi + ci_dI2_Pi
        else:
            new_const = dI2 - dj_djstar
            ci_dI2_vi_Pi = P.multiply_by_constant(new_const)
            rot_H+= ci_dI2_vi_Pi
        
    rot_H+=P.multiply_by_constant(dI2)
rot_H

0.954+0.000j XXXI +
1.535+0.000j XXXX

In [390]:
LCU_rots * H * LCU_rots.conjugate

0.250+0.000j XXXI +
0.500+0.000j XXXX

In [376]:
rot_H.coeff_vec

array([0.15664785+0.j, 0.3132957 +0.j])

In [379]:
rot_H.coeff_vec + dI2*PauliwordOp({'XXXI':0.25, 'XXXX':0.5}).coeff_vec

array([0.25+0.j, 0.5 +0.j])

In [365]:
LCU_rots[:1] * PauliwordOp({'XXXX':0.5, 'XXXI':0.25}) * LCU_rots[:1].conjugate

0.093+0.000j XXXI +
0.187+0.000j XXXX

array([0.1867043 +0.j, 0.09335215+0.j])

In [355]:
dI2*P.coeff_vec

array([0.1867043 +0.j, 0.09335215+0.j])

In [370]:
rot_H.coeff_vec + dI2*PauliwordOp({'XXXX':0.5, 'XXXI':0.25}).coeff_vec

array([0.34335215+0.j, 0.40664785+0.j])

In [340]:
dI2 * 0.5  + rot_H.coeff_vec 

array([0.5+0.j])

In [413]:
non_I_terms

[0.000+0.580j XZZI, 0.000+0.539j XYZI]

In [427]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) # PauliwordOp({'XXXX': 0.3})

#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
        rot_H+= vi_Pi 
        
        for l, P_rot2 in enumerate(non_I_terms):
            if j!=l:
                if not P_rot2.commutes(P):                    
                    # ci_dj_PjkPi
                    ci_dj_PjkPi = P_rot2 * P 
                    
                     # 2dI_ci_dj_PjkPi
                    rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
rot_H

0.250+0.000j XXXI +
-0.127+0.000j YIII +
-0.354+0.000j ZZZI +
-0.329+0.000j ZYZI

In [454]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) # PauliwordOp({'XXXX': 0.3})

#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
        rot_H+= vi_Pi 
        
        for l, P_rot2 in enumerate(non_I_terms):
            if j!=l:
                if not P_rot2.commutes(P):                    
                    # ci_dj_PjkPi
                    ci_dj_PjkPi = P_rot2 * P 

                     # 2dI_ci_dj_PjkPi
                    rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
rot_H

0.250+0.000j XXXI +
-0.127+0.000j YIII +
-0.354+0.000j ZZZI +
-0.329+0.000j ZYZI

In [438]:
LCU_rots * H * LCU_rots.conjugate

0.250+0.000j XXXI +
-0.127+0.000j YIII +
-0.354+0.000j ZZZI +
-0.329+0.000j ZYZI

In [455]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) # PauliwordOp({'XXXX': 0.3})

#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
        rot_H+= vi_Pi 
        
rot_H

0.250+0.000j XXXI +
-0.127+0.000j YIII

In [736]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) # PauliwordOp({'XXXX': 0.3})
 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
        rot_H+= vi_Pi 
        
        for l, P_rot2 in enumerate(non_I_terms):

            if not P_rot2.commutes(P):                    
                # ci_dj_PjkPi
                ci_dj_PjkPi = P_rot2 * P 

                 # 2dI_ci_dj_PjkPi
                rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI/2)
rot_H

0.250+0.000j XXXI +
-0.127+0.000j YIII +
-0.354+0.000j ZZZI +
-0.329+0.000j ZYZI

In [1090]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


# H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) 
# H =PauliwordOp({'YIII':0.5, 'XXXI':0.25, 'XYZI':2})
H =PauliwordOp({'XYZI':2})
 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)

        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
            if P.commutes(Plk):
                Pjk_Plk_Pi = P_rot * Plk*P
                rot_H+= Pjk_Plk_Pi.multiply_by_constant(-2)
            
        

        
rot_H

1.418+0.000j IXII +
1.250-0.000j XZZI +
0.654+0.000j XYZI

In [1084]:
0.580j*-1j * 2.16

(1.2528+0j)

In [1085]:
out2 = LCU_rots * H * LCU_rots.conjugate
out2

1.418+0.000j IXII +
1.250+0.000j XZZI +
0.654+0.000j XYZI

In [746]:
out2-rot_H

1.250+0.000j XZZI

In [648]:
LCU_rots

0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [920]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


# H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) 
# H =PauliwordOp({'YIII':0.5, 'XXXI':0.25, 'XYZI':2})
H =PauliwordOp({'XYZI':2})
 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
            
            
        rot_H+= vi_Pi 
        
rot_H

1.418+0.000j IXII +
0.654+0.000j XYZI

In [924]:
P_rot * P 

0.000+1.077j IIII

In [922]:
out2 = LCU_rots * H * LCU_rots.conjugate
out2

1.418+0.000j IXII +
1.250+0.000j XZZI +
0.654+0.000j XYZI

In [921]:
H.multiply_by_constant(dI2) * LCU_rots 

0.000+0.402j IIII +
-0.433+0.000j IXII +
0.456+0.000j XYZI

In [918]:
for P_RRR in LCU_rots:
    if P_RRR.commutes(P):
        print(P_RRR*P)

1.222+0.000j XYZI
0.000+1.077j IIII


In [915]:
LCU_rots[2] * H * LCU_rots[1] - LCU_rots[1] * H * LCU_rots[2]

In [874]:
for j, P_rot in enumerate(non_I_terms):
    for P in H:
#         dI_ci_dj_Pjk_Pi = (P_rot*P).multiply_by_constant(I_term_LCU.coeff_vec[0])
        
        print(P_rot)

0.000+0.580j XZZI
0.000+0.539j XYZI


In [778]:
H.multiply_by_constant(dI2)

0.747+0.000j XYZI

In [890]:
LCU_rots * H #* 
LCU_rots.conjugate
H

2.000+0.000j XYZI

In [884]:
1.077j * -0.580j + 1.077j * -0.580j

(1.2493199999999998-0j)

In [876]:
out2 = LCU_rots * H * LCU_rots.conjugate
out2

1.418+0.000j IXII +
1.250+0.000j XZZI +
0.654+0.000j XYZI

In [865]:
out2 - rot_H

1.250+0.000j XZZI

In [839]:
LCU_rots

0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [840]:
LCU_rots.conjugate

0.611-0.000j IIII +
0.000-0.580j XZZI +
0.000-0.539j XYZI

In [989]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


# H =PauliwordOp({'YIII':0.5, 'XXXI':0.25}) 
# H =PauliwordOp({'YIII':0.5, 'XXXI':0.25, 'XYZI':2})
H =PauliwordOp({'XYZI':2})
 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
            
            
        rot_H+= vi_Pi 
        
        if sum((P *P_rot).symp_matrix[0]) == 0:
            # identity term!
            for l, P_rot_l in enumerate(non_I_terms): 
                if l!=j:
                    term = P_rot_l.conjugate * P * P_rot
                    rot_H+= term 
#                     print(term)

        if sum((P_rot*P).symp_matrix[0]) == 0:
            # identity term!
            for l, P_rot_l in enumerate(non_I_terms): 
                if l!=j:
                    term = P_rot_l.conjugate * P * P_rot
                    rot_H+= term 
                    print(term)
                    
        
rot_H

0.625+0.000j XZZI


1.418+0.000j IXII +
1.250+0.000j XZZI +
0.654+0.000j XYZI

In [981]:
rot_H.n_terms# + rot_H

23

In [960]:
LCU_rots

0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [988]:
LCU_rots * H * LCU_rots.conjugate

1.418+0.000j IXII +
1.250+0.000j XZZI +
0.654+0.000j XYZI

In [993]:
LCU_rots[1:]

0.000+0.580j XZZI +
0.000+0.539j XYZI

In [1007]:
1j*-1j

(1-0j)

In [1005]:
t = PauliwordOp({'XX': 1j, 'XY': 1j} )
PP = PauliwordOp({'XY':1, 'ZI':2})

t*PP*t.conjugate

2.000+0.000j XX +
-4.000+0.000j ZI

In [1017]:
LCU_rots

0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [1018]:
LCU_rots.conjugate

0.611-0.000j IIII +
0.000-0.580j XZZI +
0.000-0.539j XYZI

In [1021]:
t = PauliwordOp({'XX': 1j} )
y = PauliwordOp({'XY': 1j} )
y*t

0.000+1.000j IZ

In [1022]:
for i in range(3):
    for j in range(3):
        if j!=i:
            print(i,j)

0 1
0 2
1 0
1 2
2 0
2 1


In [1030]:
LCU_rots

0.611+0.000j IIII +
0.000+0.580j XZZI +
0.000+0.539j XYZI

In [1233]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
            if (P.commutes(Plk) and  (not P.commutes(P_rot))):
                
#                 print(l,j)
                
                Pjk_Plk_Pi =  Plk*P_rot*P - P_rot*Plk*P
                rot_H+= Pjk_Plk_Pi
                
            elif (not P.commutes(Plk) and   P.commutes(P_rot)):
                
#                 print(l,j)
                
                Pjk_Plk_Pi =  P_rot*Plk*P - Plk*P_rot*P
                rot_H+= Pjk_Plk_Pi
            
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1232]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
            if (P.commutes(Plk) and  (not P.commutes(P_rot))):
                
                
                Pi_Pjk_Pkl =  P * P_rot * Plk
                Pi_Plk_Pkj =  P * Plk * P_rot.conjugate
#                 print('t:', Pi_Pjk_Pkl+Pi_Plk_Pkj)
#                 print(j,l)
                
                rot_H+= Pi_Pjk_Pkl
                rot_H+= Pi_Plk_Pkj
                
            elif (not P.commutes(Plk) and   P.commutes(P_rot)):
                
                Pi_Pjk_Pkl =  P * P_rot.conjugate * Plk
                Pi_Plk_Pkj =  P * Plk * P_rot
                
#                 print('t:', Pi_Pjk_Pkl+Pi_Plk_Pkj)
                
                rot_H+= Pi_Pjk_Pkl
                rot_H+= Pi_Plk_Pkj
            
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1236]:
# empty_I = 'I'*H.n_qubits
# rot_H = PauliwordOp({empty_I: 0})
# dI2 = I_term_LCU.coeff_vec[0]**2
# two_dI = 2*I_term_LCU.coeff_vec[0]

 
# #equation B49
# rot_H = H.multiply_by_constant(dI2) # first term!
# for j, P_rot in enumerate(non_I_terms):
#     dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
#     for P in H:
#         if P.commutes(P_rot):
#             #vi = ci_di_di*
#             vi_Pi = P.multiply_by_constant(dj_djstar)
#         else:
#             vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
# #             # extra term from anticommuting part!
# #             ci_dj_PjkPi = P_rot * P 
# #              # 2dI_ci_dj_PjkPi
# #             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
#         rot_H+= vi_Pi 
    
 
#         if not P.commutes(P_rot):
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
#         for l in range(j+1,len(non_I_terms)):
#             Plk = non_I_terms[l]

#             if P.commutes(P_rot) and   (not P.commutes(Plk)):
                
#                 Pi_Pjk_Pkl =  P * P_rot * Plk.conjugate
#                 Pi_Plk_Pkj =  P * Plk * P_rot.conjugate
                
#                 rot_H+= Pi_Pjk_Pkl
#                 rot_H-= Pi_Plk_Pkj
            
#             elif (not P.commutes(P_rot) and P.commutes(Plk)):
                
#                 Pi_Pjk_Pkl =  P * P_rot.conjugate * Plk
#                 Pi_Plk_Pkj =  P * Plk.conjugate * P_rot
                
#                 rot_H+= Pi_Pjk_Pkl
#                 rot_H-= Pi_Plk_Pkj
            
        
        
# out1 = LCU_rots * H * LCU_rots.conjugate
# out1 - rot_H

-0.875+0.000j XYXI +
-0.875+0.000j ZYZI +
0.125+0.000j ZZYI

In [1266]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = PauliwordOp({'IIII':0})
for j, P_rot_j in enumerate(non_I_terms):
    for P in H:
        for l, P_rot_l in enumerate(non_I_terms):
            
            if j==l:
                if (P.commutes(P_rot_j)):
                    rot_H+=P_rot_j*P*P_rot_l.conjugate
                elif (not P.commutes(P_rot_l)):
                    rot_H+=P_rot_j*P*P_rot_l.conjugate
                
            
rot_H.n_terms == H.n_terms

True

In [1291]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = PauliwordOp({'IIII':0})
for j, P_rot_j in enumerate(non_I_terms):
    for P in H:
        for l, P_rot_l in enumerate(non_I_terms):
            
            if j==l:
                if (P.commutes(P_rot_j)):
                    rot_H+=P_rot_j*P*P_rot_l.conjugate
                elif (not P.commutes(P_rot_l)):
                    rot_H+=P_rot_j*P*P_rot_l.conjugate
            
            if j!=l:
                if (P.commutes(P_rot_j)):
                    rot_H+=P_rot_j*P*P_rot_l.conjugate
                elif (not P.commutes(P_rot_l)):
                    rot_H+=P_rot_j*P*P_rot_l.conjugate
                
                
            
rot_H.n_terms == (LCU_rots[1:] *H*LCU_rots[1:].conjugate).n_terms
rot_H - (LCU_rots[1:] *H*LCU_rots[1:].conjugate)

0.219+0.000j XZXI +
0.219+0.000j XYXI +
-0.031+0.000j YYXI +
-0.187+0.000j XYZI +
0.062+0.000j YZZI +
0.219+0.000j ZYZI +
-0.031+0.000j ZZYI

In [ ]:
# out1 = LCU_rots * H * LCU_rots.conjugate
# out1 - rot_H

In [1313]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
#             Pkl = non_I_terms[l].conjugate
            if (P.commutes(P_rot) and  (not P.commutes(Plk))):
                
                
                Pi_Pjk_Plk =  P * P_rot * Plk.conjugate
                
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
                
            elif P.commutes(Plk) and   (not P.commutes(P_rot)):
                
                Pi_Pjk_Plk =  P *  Plk.conjugate * P_rot
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
            
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1309]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
        if P.commutes(P_rot):
            #vi = ci_di_di*
            vi_Pi = P.multiply_by_constant(dj_djstar)
        else:
            vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
#             # extra term from anticommuting part!
#             ci_dj_PjkPi = P_rot * P 
#              # 2dI_ci_dj_PjkPi
#             rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)    
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
            if (P.commutes(P_rot) and  (not P.commutes(Plk))):
                
                Pi_Pjk_Plk =  P * P_rot * Plk.conjugate
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
                
                Plk_fake = non_I_terms[j].conjugate
                Pjk_fake = non_I_terms[l]
                Pi_Pjk_Plk =  P *  Pjk_fake * Pjk_fake
                print(Pi_Pjk_Plk)
                
                
            elif P.commutes(Plk) and   (not P.commutes(P_rot)):
                
                Pi_Pjk_Plk =  P *  Plk.conjugate * P_rot
                print(Pi_Pjk_Plk)
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
            
            print()
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H


-0.203+0.000j XYXI

-0.219+0.000j XYXI

-0.174+0.000j XZZI


-0.219+0.000j ZYZI






-0.058+0.000j YYZI

-0.029+0.000j YZXI

0.031-0.000j ZZYI



In [1357]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
#         if P.commutes(P_rot):
#             #vi = ci_di_di*
#             vi_Pi = P.multiply_by_constant(dj_djstar)
#         else:
#             vi_Pi = P.multiply_by_constant(-1*dj_djstar)
            
        vi_Pi = P_rot * P * P_rot.conjugate 
        
        rot_H+= vi_Pi 
    
 
        if not P.commutes(P_rot):
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_rot * P 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
#             Pkl = non_I_terms[l].conjugate
            if (P.commutes(P_rot) and  (not P.commutes(Plk))):
                
                Pi_Pjk_Plk =  P * P_rot * Plk.conjugate
                
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
                
            elif P.commutes(Plk) and   (not P.commutes(P_rot)):
                
                Pi_Pjk_Plk =  P *  Plk.conjugate * P_rot
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
            
            
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1331]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

In [1344]:
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:
#         if P.commutes(P_rot):
#             #vi = ci_di_di*
#             vi_Pi = P.multiply_by_constant(dj_djstar)
#         else:
#             vi_Pi = P.multiply_by_constant(-1*dj_djstar)
#         rot_H+= vi_Pi 
        
        rot_H+= P_rot * P * P_rot.conjugate
    

rot_H - temp

In [1342]:
temp = deepcopy(rot_H)

In [1329]:
%%timeit
out1 = LCU_rots * H * LCU_rots.conjugate

3.48 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1368]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_rot in enumerate(non_I_terms):
    dj_djstar = P_rot.coeff_vec[0]*P_rot.coeff_vec[0].conj()
    for P in H:        
        for l in range(j+1,len(non_I_terms)):
            Plk = non_I_terms[l]
            if (P.commutes(P_rot) and  (not P.commutes(Plk))):
                
                
                Pi_Pjk_Plk =  P * P_rot * Plk.conjugate
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
                print(Pi_Pjk_Plk.multiply_by_constant(2))
                
            elif P.commutes(Plk) and   (not P.commutes(P_rot)):
                
                Pi_Pjk_Plk =  P *  Plk.conjugate * P_rot
                rot_H+= Pi_Pjk_Plk.multiply_by_constant(2)
                
                print(Pi_Pjk_Plk.multiply_by_constant(2))
            
            else:
                continue
            print()
        
# rot_H

-0.437+0.000j XZXI

-0.437+0.000j XYXI

0.375+0.000j XYZI

-0.437+0.000j ZYZI

-0.125+0.000j YZZI

0.062+0.000j YYXI

0.062+0.000j ZZYI



In [1375]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:
        vi_Pi = P_jk * ci_Pi * P_jk.conjugate 
        rot_H+= vi_Pi 
    
        anti_comm_Pi_Pjk = not ci_Pi.commutes(P_jk)
        if anti_comm_Pi_Pjk:
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_jk * ci_Pi 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            P_lk = non_I_terms[l]
            comm_Pi_Plk = ci_Pi.commutes(P_lk)
            if ( (not anti_comm_Pi_Pjk) and  (not comm_Pi_Plk)):
                
                Pi_Pjk_Pkl =  ci_Pi * P_jk * Plk.conjugate
                rot_H+= Pi_Pjk_Pkl.multiply_by_constant(2)
                
            elif (anti_comm_Pi_Pjk and comm_Pi_Plk):
                
                Pi_Pkl_Pjk =  ci_Pi *  Plk.conjugate * P_jk
                rot_H+= Pi_Pkl_Pjk.multiply_by_constant(2)

        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1381]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:
        vi_Pi = P_jk * ci_Pi * P_jk.conjugate 
        rot_H+= vi_Pi 
    
        anti_comm_Pi_Pjk = not ci_Pi.commutes(P_jk)
        if anti_comm_Pi_Pjk:
            # extra term from anticommuting part!
            ci_dj_PjkPi = P_jk * ci_Pi 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l in range(j+1,len(non_I_terms)):
            P_lk = non_I_terms[l]
            comm_Pi_Plk = ci_Pi.commutes(P_lk)
            if ( (not anti_comm_Pi_Pjk) and  (not comm_Pi_Plk)):
                
                Pi_Pjk_Pkl =  ci_Pi * P_jk * Plk.conjugate
                rot_H+= Pi_Pjk_Pkl.multiply_by_constant(2)
                
            elif (anti_comm_Pi_Pjk and comm_Pi_Plk):
                
                Pi_Pkl_Pjk =  ci_Pi *  Plk.conjugate * P_jk
                rot_H+= Pi_Pkl_Pjk.multiply_by_constant(2)

        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1386]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:
        vi_Pi = P_jk * ci_Pi * P_jk.conjugate 
        rot_H+= vi_Pi 
    
        
        # anti-comm first
        P_kl = P_jk.conjugate
        Pi_Pkl = (ci_Pi*P_kl).multiply_by_constant(I_term_LCU.coeff_vec[0])
        Pjk_Pi = (P_jk*ci_Pi).multiply_by_constant(I_term_LCU.coeff_vec[0])
        
        rot_H+= (Pi_Pkl+Pjk_Pi)
        
        
        for l in range(j+1,len(non_I_terms)):
            P_lk = non_I_terms[l]
            comm_Pi_Plk = ci_Pi.commutes(P_lk)
            if ( (not anti_comm_Pi_Pjk) and  (not comm_Pi_Plk)):
                
                Pi_Pjk_Pkl =  ci_Pi * P_jk * Plk.conjugate
                rot_H+= Pi_Pjk_Pkl.multiply_by_constant(2)
                
            elif (anti_comm_Pi_Pjk and comm_Pi_Plk):
                
                Pi_Pkl_Pjk =  ci_Pi *  Plk.conjugate * P_jk
                rot_H+= Pi_Pkl_Pjk.multiply_by_constant(2)

        
# out1 = LCU_rots * H * LCU_rots.conjugate
# out1 - rot_H

In [1429]:
#equation B49
rot_H = PauliwordOp({'IIII':0})
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:    
        # anti-comm first
        P_kl = P_jk.conjugate
        Pi_Pkl = (ci_Pi*P_kl).multiply_by_constant(I_term_LCU.coeff_vec[0])
        Pjk_Pi = (P_jk*ci_Pi).multiply_by_constant(I_term_LCU.coeff_vec[0])
        
        rot_H+= (Pi_Pkl+Pjk_Pi)

## vs

left_mult = (LCU_rots[1:]*H)#.multiply_by_constant(I_term_LCU.coeff_vec[0])
righ_mult = (H*LCU_rots[1:].conjugate)#.multiply_by_constant(I_term_LCU.coeff_vec[0])
rot_H - (left_mult + righ_mult).multiply_by_constant(I_term_LCU.coeff_vec[0])

In [1489]:

dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]

empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0}) 
#equation B49
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:
        vi_Pi = P_jk * ci_Pi * P_jk.conjugate 
        rot_H+= vi_Pi 
    
        
        # anti-comm first
        P_kl = P_jk.conjugate
        Pi_Pkl = (ci_Pi*P_kl).multiply_by_constant(I_term_LCU.coeff_vec[0])
        Pjk_Pi = (P_jk*ci_Pi).multiply_by_constant(I_term_LCU.coeff_vec[0])
        
        rot_H+= (Pi_Pkl+Pjk_Pi)
        
        
        for l, P_lk in enumerate(non_I_terms[(j+1):]):
            P_jk_Pi_P_kl = P_jk * ci_Pi * P_lk.conjugate
            P_lk_Pi_P_kj = P_lk * ci_Pi * P_jk.conjugate
            rot_H+= (P_jk_Pi_P_kl+P_lk_Pi_P_kj)
            
#             if ( (not ci_Pi.commutes(P_lk)) and  (not ci_Pi.commutes(P_jk))):
#                 print('emtpy:', (P_jk_Pi_P_kl+P_lk_Pi_P_kj))
#             if ( ( ci_Pi.commutes(P_lk)) and  ( ci_Pi.commutes(P_jk))):
#                 print('emtpy:', (P_jk_Pi_P_kl+P_lk_Pi_P_kj))

        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1573]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


In [1574]:
%%timeit
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:
        vi_Pi = P_jk * ci_Pi * P_jk.conjugate 
        rot_H+= vi_Pi 
    
        
        # anti-comm first
        P_kl = P_jk.conjugate
        Pi_Pkl = (ci_Pi*P_kl).multiply_by_constant(I_term_LCU.coeff_vec[0])
        Pjk_Pi = (P_jk*ci_Pi).multiply_by_constant(I_term_LCU.coeff_vec[0])
        
        rot_H+= (Pi_Pkl+Pjk_Pi)
        
        
        for l, P_lk in enumerate(non_I_terms[(j+1):]):
            P_jk_Pi_P_kl = P_jk * ci_Pi * P_lk.conjugate
            P_lk_Pi_P_kj = P_lk * ci_Pi * P_jk.conjugate
            rot_H+= (P_jk_Pi_P_kl+P_lk_Pi_P_kj)
            

        



32.3 ms ± 408 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1474]:
%%timeit
out1 = LCU_rots * H * LCU_rots.conjugate


3.36 ms ± 49.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1486]:
#equation B49
rot_H = PauliwordOp({'IIII':0})
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:    
        for l, P_lk in enumerate(non_I_terms[(j+1):]):
            P_jk_Pi_P_kl = P_jk * ci_Pi * P_lk.conjugate
            P_lk_Pi_P_kj = P_lk * ci_Pi * P_jk.conjugate
            rot_H+= (P_jk_Pi_P_kl+P_lk_Pi_P_kj)

## vs

left_mult = (LCU_rots[1:]*H)#.multiply_by_constant(I_term_LCU.coeff_vec[0])
righ_mult = (H*LCU_rots[1:].conjugate)#.multiply_by_constant(I_term_LCU.coeff_vec[0])

test = PauliwordOp({'IIII':0})
for j, P_jk in enumerate(non_I_terms):
    for ci_Pi in H:    
        l=(j+1)
        ci_P_jk_Pi_P_kl = P_jk * righ_mult[l:]
        bbbbb = P_jk * righ_mult[l:].conjugate
        
        test +=(ci_P_jk_Pi_P_kl+bbbbb)
        
rot_H-test

0.000+0.000j YIXI +
3.509-0.000j YXXI +
12.281+0.000j XZXI +
11.373+0.000j XYXI +
0.812-0.000j YZXI +
-0.812+0.000j YYXI +
0.000+0.000j XIZI +
3.509-0.000j XXZI +
10.527-0.000j IIYI +
1.754-0.000j YXYI +
-4.874+0.000j XYZI +
6.595-0.000j ZZZI +
1.625+0.000j YZZI +
5.686+0.000j ZYZI +
1.625-0.000j YYZI +
-0.812+0.000j ZZYI +
0.942-0.000j ZYYI

0.000+0.580j XZZI +
0.000+0.539j XYZI

1

In [1567]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


non_I_terms = LCU_rots[1:]

commutation_check = H.commutes_termwise(non_I_terms)
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for i, ci_Pi in enumerate(H):
        
        # vi_Pi = P_jk * ci_Pi * P_jk.conjugate
        # rot_H += vi_Pi
        signed_bjk_blk_2 = (-1) ** (not commutation_check[i, j]) * P_jk.coeff_vec[0] * P_jk.coeff_vec[0].conjugate()
        vi_Pi = ci_Pi.multiply_by_constant(signed_bjk_blk_2)
        rot_H += vi_Pi
        
    
 
        if not commutation_check[i, j]: #ci_Pi.commutes(P_rot)            
            ci_dj_PjkPi = P_jk * ci_Pi 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l, P_lk in enumerate(non_I_terms[(j+1):]):
            l_ind = l+(j+1)
            if (commutation_check[i,j] and  (not commutation_check[i,l_ind])):
                Pi_Pjk_Pkl =  ci_Pi * P_jk * Plk.conjugate           
                rot_H+= Pi_Pjk_Pkl.multiply_by_constant(2)

            elif ((not commutation_check[i,j]) and  (commutation_check[i,l_ind])):
                Pi_Pkl_Pjk =  ci_Pi *  Plk.conjugate * P_jk
                rot_H+= Pi_Pkl_Pjk.multiply_by_constant(2)
            
        
        
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1568]:
empty_I = 'I'*H.n_qubits
rot_H = PauliwordOp({empty_I: 0})
dI2 = I_term_LCU.coeff_vec[0]**2
two_dI = 2*I_term_LCU.coeff_vec[0]


non_I_terms = LCU_rots[1:]

In [1578]:
%%timeit
commutation_check = H.commutes_termwise(non_I_terms)
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for i, ci_Pi in enumerate(H):
        
        # vi_Pi = P_jk * ci_Pi * P_jk.conjugate
        # rot_H += vi_Pi
        signed_bjk_blk_2 = (-1) ** (not commutation_check[i, j]) * P_jk.coeff_vec[0] * P_jk.coeff_vec[0].conjugate()
        vi_Pi = ci_Pi.multiply_by_constant(signed_bjk_blk_2)
        rot_H += vi_Pi
        
    
 
        if not commutation_check[i, j]: #ci_Pi.commutes(P_rot)            
            ci_dj_PjkPi = P_jk * ci_Pi 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l, P_lk in enumerate(non_I_terms[(j+1):]):
            l_ind = l+(j+1)
            if (commutation_check[i,j] and  (not commutation_check[i,l_ind])):
                Pi_Pjk_Pkl =  ci_Pi * P_jk * Plk.conjugate           
                rot_H+= Pi_Pjk_Pkl.multiply_by_constant(2)

            elif ((not commutation_check[i,j]) and  (commutation_check[i,l_ind])):
                Pi_Pkl_Pjk =  ci_Pi *  Plk.conjugate * P_jk
                rot_H+= Pi_Pkl_Pjk.multiply_by_constant(2)
            
        

12.3 ms ± 623 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1579]:
commutation_check = H.commutes_termwise(non_I_terms)
rot_H = H.multiply_by_constant(dI2) # first term!
for j, P_jk in enumerate(non_I_terms):
    for i, ci_Pi in enumerate(H):
        
        # vi_Pi = P_jk * ci_Pi * P_jk.conjugate
        # rot_H += vi_Pi
        signed_bjk_blk_2 = (-1) ** (not commutation_check[i, j]) * P_jk.coeff_vec[0] * P_jk.coeff_vec[0].conjugate()
        vi_Pi = ci_Pi.multiply_by_constant(signed_bjk_blk_2)
        rot_H += vi_Pi
        
    
 
        if not commutation_check[i, j]: #ci_Pi.commutes(P_rot)            
            ci_dj_PjkPi = P_jk * ci_Pi 
             # 2dI_ci_dj_PjkPi
            rot_H+= ci_dj_PjkPi.multiply_by_constant(two_dI)
        
        for l, P_lk in enumerate(non_I_terms[(j+1):]):
            l_ind = l+(j+1)
            if (commutation_check[i,j] and  (not commutation_check[i,l_ind])):
                Pi_Pjk_Pkl =  ci_Pi * P_jk * Plk.conjugate           
                rot_H+= Pi_Pjk_Pkl.multiply_by_constant(2)

            elif ((not commutation_check[i,j]) and  (commutation_check[i,l_ind])):
                Pi_Pkl_Pjk =  ci_Pi *  Plk.conjugate * P_jk
                rot_H+= Pi_Pkl_Pjk.multiply_by_constant(2)
                
out1 = LCU_rots * H * LCU_rots.conjugate
out1 - rot_H

In [1575]:
Pjk_Pi = P_jk._multiply_single_Pword_phaseless(ci_Pi)
c

1.000+0.000j YIXI

In [1601]:

rot_H = H.copy()
for R_sk, angle in cs_vqe_obj.unitary_partitioning_rotations:
    for P in H:
        rot_H= (rot_H._rotate_by_single_Pword(R_sk, angle)).cleanup()
        
rot_H

-0.020+0.000j IXII +
0.100+0.000j XXXI +
-0.007+0.000j ZIII +
-0.700+0.000j YIII +
0.097+0.000j YXXI +
0.182+0.000j IZII +
-0.083+0.000j IYII +
0.910+0.000j XZXI +
0.255+0.000j XYXI +
0.091+0.000j ZZXI +
-0.042+0.000j YZXI +
-0.041+0.000j ZYXI +
-0.091+0.000j YYXI +
-0.003+0.000j XXZI +
0.668+0.000j IIYI +
0.500+0.000j XXYI +
0.003+0.000j ZXYI +
-0.001+0.000j YXYI +
-0.249+0.000j XZZI +
-0.546+0.000j XYZI +
0.025+0.000j ZZZI +
0.182+0.000j YZZI +
-0.083+0.000j YYZI +
-0.182+0.000j ZZYI +
0.087+0.000j ZYYI +
0.500+0.000j IIIZ

In [1611]:
from symred.utils import get_ground_state_sparse

In [1618]:
LCU_mat = get_ground_state_sparse((LCU_rots * H * LCU_rots.conjugate).to_sparse_matrix)
seq_rot_mat = get_ground_state_sparse(rot_H.to_sparse_matrix)
                                     
LCU_mat[0] - seq_rot_mat[0]

4.884981308350689e-15